# Final Report

In this final lesson we are going to put together our final excel sheet reporting. It will implement everything from data to creating the two report sheets. 

## Getting Started

Firstly we are going to grab all our prior defined functions and add a few more for convenience.

In [1]:
import pandas as pd
from datetime import datetime
from matplotlib.ticker import PercentFormatter
import matplotlib.pyplot as plt
import openpyxl

from openpyxl.styles.alignment import Alignment
from openpyxl.styles import Font
from openpyxl.styles import PatternFill
from openpyxl.utils import get_column_letter
from openpyxl.utils.cell import coordinate_from_string, column_index_from_string
from openpyxl.styles import Border, Side
from openpyxl.drawing.image import Image
from typing import List, Tuple
from pandas import DataFrame 
from openpyxl.worksheet.worksheet import Worksheet
from copy import copy

# Redefine all functions

def create_yield_table(data: DataFrame, date: datetime) -> DataFrame:
    """
    Function to build a yield table object

    Parameters
    ----------
    data : DataFrame
        Yield curve data to be used
    date : datetime
        The date to use for the current date

    Yields
    ------
    DataFrame
        A yield table

    """
    # Find the other two dates
    date2 = date.replace(year=date.year-1)
    date3 = date.replace(year=date.year-5)

    # Index into the three dates and copy
    table = yield_curve.loc[[date, date2, date3]].copy()

    # Transpose
    table = table.T

    # Rename the columns
    table.columns = ["Current", "1 Year Ago Curve", "5 Years Ago Curve"]

    # Get the changes in the yield curve
    table["1 Year Change"] = table["Current"] - table["1 Year Ago Curve"]
    table["5 Year Change"] = table["Current"] - table["5 Years Ago Curve"]

    return table

def create_title(ws: Worksheet, area: str, text: str) -> None:
    """
    Writes a title on to the passed worksheet

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    area : str
        The area to put the title in (of the form A1:A5 or similar)
    text : str
        The text for the title

    """
    # Merge the cells in the area
    ws.merge_cells(area)
    
    # Given an area such as A1:A5, we want to first isolate the first cell
    # Split will break the area into the two components, we take the first one
    first_cell = area.split(":")[0]
    
    # Find the coordinates
    first_cell = coordinate_from_string(first_cell)
    
    # Convert the components to the numerical values for column and row
    col = column_index_from_string(first_cell[0])
    row = first_cell[1]
    
    # Change the text of the merged cell to whatever was passed as input
    ws.cell(row=row, column=col).value = text
    
    # Change the font of the title
    font = font = Font(size=16,
                       bold=True,
                      color="283747")
    ws.cell(row=row, column=col).font = font
    
    # Change alignment to be centered
    ws.cell(row=row, column=col).alignment = Alignment(horizontal="center")
    
def convert_to_year_frac(label: str) -> float:
    """
    Function to convert treasury curve indices to floating
    point numbers

    Parameters
    ----------
    label : str
        The string index label.

    Returns
    -------
    float
        A floating point number representing a fraction of a year.

    """
    
    # Handle months
    if "Mo" in label:
        # Get rid of string part
        frac = label.replace(' Mo','')
        
        # Convert to a fraction over 12
        frac = float(frac)
        frac = frac/12
    # Handle years
    else:
        # Get rid of string part
        frac = label.replace(' Yr','')
        # Convert to floating point number
        frac = float(frac)
    
    return frac

def write_to_excel(ws: Worksheet, start_row: int,
                   start_column: int, df: DataFrame,
                   index: bool = True, columns: bool = True) -> None:
    """
    Function which takes a worksheet and writes a table of data to it.

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start_row : int
        The row to begin on
    start_column : int
        The column to start on
    df : DataFrame
        The data to write
    index : bool, optional
        Whether to write the index labels. The default is True.
    columns : bool, optional
        Whether to write the column labels. The default is True.

    """
    
    # If we want to include the index, reset the index on the dataframe first
    if index:
        write_data = df.reset_index().values.tolist()
    else:
        write_data = df.values.tolist()
    
    # If we want to include columns we need to add the columns as the first in the nested lists
    if columns:
        # If we already added an index, then we are going to want also include a blank space since
        # we don't want a label above the index
        if index:
            write_data = [[""]+list(df.columns)]+write_data
        else:
            write_data = [list(df.columns)]+write_data
    
    # For every value iterate through finding the location to place it in
    for row in range(len(write_data)):
        for column in range(len(write_data[0])):
            ws.cell(row=start_row+row,column=start_column+column).value = write_data[row][column]
            
def rgb_to_hex(rgb: tuple) -> str:
    """
    This function takes a RGB color tuple and returns back
    a hexadecimal version of it.

    Parameters
    ----------
    rgb : tuple
        A tuple of length three with values between 0-255 for each. It
        represents red, green and blue values.

    Returns
    -------
    str
        The hexadecimal representation

    """
    return '%02x%02x%02x' % rgb

def adjust_column_widths(ws: Worksheet, start: int, values: List[int]):
    """
    Function to adjust the width of the columns starting at the start value
    to the values listed in values

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start : int
        The column to begin adjustments at
    values : List[int]
        The widths to set each column to in order


    """
    # Iterate through the values
    for i, width in enumerate(values):
        # Set the width of the column
        ws.column_dimensions[get_column_letter(start+i)].width = width
        
def value_to_red_green(value: float) -> tuple:
    """
    Convert a value ranging from [-1, 1] to red or green with
    intensity based on the value

    Parameters
    ----------
    value : float
        The value ranging from -1 to 1

    Returns
    -------
    tuple
        The RGB tuple

    """
    # Positive values are green
    if value > 0:
        return (int(255-value*255), 255, int(255-value*255))
    # Negative values are red
    else:
        value = abs(value)
        return (255, int(255-value*255), int(255-value*255))

def create_color_scale(returns: DataFrame) -> DataFrame:
    """
    Function to build a color scale where each value is transformed
    into hexadecimal colors based on a scaling where more red is more
    negative and more green is more positive.

    Parameters
    ----------
    returns : DataFrame
        Returns data to start with

    Returns
    -------
    DataFrame
        The color data
    """
    # Find the maximum returns
    max_abs_return = abs(returns).max().max()
    
    # Divide each number by this
    scale = returns / max_abs_return
    
    # Map the conversion to rgb
    scale = scale.applymap(value_to_red_green)
    
    # Map rgb to hex
    scale = scale.applymap(rgb_to_hex)
    
    return scale

def red_green_map(ws: Worksheet, returns: DataFrame, start_row: int, start_column: int) -> None:
    """
    Function which, given a DataFrame, maps the colors to red and
    green based on how positive/negative values are.

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    returns : DataFrame
        The returns data
    start_row : int
        The starting row
    start_column : int
        The starting column

    """
    # Create the color scale
    color_scale = create_color_scale(returns.fillna(0))
    
    # Iterate through rows and columns adding the colors
    for row in range(returns.shape[0]):
        for column in range(returns.shape[1]):
            # Define the color
            color = PatternFill(start_color=color_scale.iloc[row, column],
                   end_color=color_scale.iloc[row, column],
                   fill_type='solid')
            
            # Set the colors
            ws.cell(row=start_row+row,column=start_column+column).fill = color
            
def bold_cells_line(ws: Worksheet, start_row: int, start_column: int,
                    length: int, vertical: bool = True) -> None:
    """
    Function which bolds a column or row of a worksheet

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start_row : int
        The row to start at for bolding
    start_column : int
        The column to start at for bolding
    length : int
        The length of the column or row to bold through
    vertical : bool, optional
        Decides if it is a row or column of the parameter length
        that is bolded. The default is True.

    """
    
    # Iterate through the length of the row/column
    for i in range(length):
        # Increment the row or column depending on if vertical is true
        if vertical:
            cell = ws.cell(row=start_row+i,column=start_column)
        else:
            cell = ws.cell(row=start_row,column=start_column+i)
            
        # Copy the font and set the bold property to true
        font = copy(cell.font)
        font.bold = True
        cell.font = font
        
def create_axis_labels(ws: Worksheet, returns: DataFrame):
    """
    Function which adds axis labels to the worksheet

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    returns: DataFrame
        The returns data

    """
    # Merge the cells and label it year
    ws.merge_cells(start_row=4, start_column=1, end_row=4+returns.shape[0], end_column=1)
    ws.cell(row=4,column=1).value = "Year"
    
    # Add alignment and font
    ws.cell(row=4, column=1).alignment = Alignment(horizontal="center", vertical="center", textRotation=90)
    ws.cell(row=4, column=1).font = Font(size=16, bold=True, color="283747")
    
    # Merge the cells and label it quarter
    ws.merge_cells("B3:F3")
    ws.cell(row=3,column=2).value = "Quarter"
    
    # Add alignment and font
    ws.cell(row=3, column=2).font = Font(size=16, bold=True, color="283747")
    ws.cell(row=3, column=2).alignment = Alignment(horizontal="center", vertical="center")

def create_number_formatting(ws: Worksheet, start_row: int,
                             start_column: int, vertical_length: int,
                             horizontal_length: int, num_format: str) -> None:
    """
    Function to change the number formatting of a certain area of an excel sheet.

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start_row : int
        The row to start changing the number formatting at
    start_column : int
        The column to start changing the number formatting at
    vertical_length : int
        How large the area is vertically
    horizontal_length : int
        How large the area is horizontally
    num_format : str
        The number format to apply

    """
    
    # Iterate through rows and columns and set the number format
    for row in range(vertical_length):
        for column in range(horizontal_length):
            ws.cell(row=start_row+row,column=start_column+column).number_format = num_format
            
def create_border(ws: Worksheet, start_row: int, start_column: int,
                  vertical_length: int, horizontal_length: int) -> None:
    """
    Function which takes a starting position and the length vertically
    and horizontally, then creates thin borders around all of them

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    start_row : int
        The row to begin at
    start_column : int
        The column to begin at
    vertical_length : int
        How far down borders should be applied
    horizontal_length : int
        How far right borders should be applied
    """
    # Create the border
    border = Border(left=Side(border_style='thin'),
         right=Side(border_style='thin'),
         top=Side(border_style='thin'),
         bottom=Side(border_style='thin'))
    
    # Iterate and set the border
    for row in range(vertical_length):
        for column in range(horizontal_length):
            ws.cell(row=start_row+row,column=start_column+column).border = border
            
def write_treasury_sheet(ws: Worksheet, yield_curve_table: DataFrame) -> None:
    """
    Function to fill out the treasury curve data

    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    yield_curve_table : DataFrame
        The yield curve data

    """
    # Write the yield curve table
    write_to_excel(ws, 3, 1, yield_curve_table)
    
    # Color the yield curve changes red/green
    red_green_map(ws, yield_curve_table[["1 Year Change"]], 4, 4)
    red_green_map(ws, yield_curve_table[["5 Year Change"]], 4, 6)
    
    # Adjust the column widths for the table
    adjust_column_widths(ws,2,[15]*len(yield_curve_table.columns))
    
    # Bold the index and columns
    bold_cells_line(ws, 4, 1, len(yield_curve_table), vertical=True)
    bold_cells_line(ws, 3, 2, len(yield_curve_table.columns), vertical=False)
    
    # Create title
    create_title(ws, 'A1:F1', "Treasury Yield Curve")
    
    # Add borders
    create_border(ws, 4, 2, yield_curve_table.shape[0], yield_curve_table.shape[1])
    
    # Get rid of gridlines
    ws.sheet_view.showGridLines = False
    
    # Add yield curve graphs
    img = Image('Images/Treasury Yield Curves.png')
    ws.add_image(img, 'H1')
    
def create_real_estate_report(ws: Worksheet, real_estate_returns: DataFrame, image_name: str):
    """
    Function to write the real estate report
    
    Parameters
    ----------
    ws : Worksheet
        The worksheet to modify
    real_estate_returns : DataFrame
        The returns data for real estate
    image_name : str
        The name of the image to import in and place
        in the report

    """
    
    # Write the table
    write_to_excel(ws, 4, 2, real_estate_returns)
    
    # Create the axis labels
    create_axis_labels(ws, real_estate_returns)
    
    # Create percentage number formatting
    create_number_formatting(ws, 5, 3, real_estate_returns.shape[0], real_estate_returns.shape[1], "0.0%")
    
    # Map the red-green color map
    red_green_map(ws, real_estate_returns, 5, 3)
    
    # Add image
    img = Image(image_name)
    ws.add_image(img, 'H5')

## Data Creation

### Yield Curve Data

We move to defining a function which prepares the yield curve data. It begins with first reading in the data, and then doing the prior data prep steps that we had done in the first lesson.

In [2]:
def prepare_data_yield_curve() -> DataFrame:
    """
    Function which puts together the yield curve data

    Returns
    -------
    DataFrame
        A yield curve datafram

    """
    # Read in the csv
    yield_curve = pd.read_csv("Data/Yield Curve.csv",index_col=0)
    
    # Convert the index to be datetime
    yield_curve.index = pd.to_datetime(yield_curve.index)
    
    # Drop the 2 month column and any null valued rows
    yield_curve = yield_curve.drop(columns="2 Mo")
    yield_curve = yield_curve.dropna()
    
    # Re-index to the every day and fill null values forward
    date_range = pd.date_range(yield_curve.index.min(), yield_curve.index.max())
    yield_curve = yield_curve.reindex(index=date_range)
    yield_curve = yield_curve.fillna(method='ffill')
    
    return yield_curve
    
    
yield_curve = prepare_data_yield_curve()
print(yield_curve)

            1 Mo  3 Mo  6 Mo  1 Yr  2 Yr  3 Yr  5 Yr  7 Yr  10 Yr  20 Yr  \
2001-07-31  3.67  3.54  3.47  3.53  3.79  4.06  4.57  4.86   5.07   5.61   
2001-08-01  3.65  3.53  3.47  3.56  3.83  4.09  4.62  4.90   5.11   5.63   
2001-08-02  3.65  3.53  3.46  3.57  3.89  4.17  4.69  4.97   5.17   5.68   
2001-08-03  3.63  3.52  3.47  3.57  3.91  4.22  4.72  4.99   5.20   5.70   
2001-08-04  3.63  3.52  3.47  3.57  3.91  4.22  4.72  4.99   5.20   5.70   
...          ...   ...   ...   ...   ...   ...   ...   ...    ...    ...   
2020-03-14  0.33  0.28  0.38  0.38  0.49  0.58  0.70  0.89   0.94   1.31   
2020-03-15  0.33  0.28  0.38  0.38  0.49  0.58  0.70  0.89   0.94   1.31   
2020-03-16  0.25  0.24  0.29  0.29  0.36  0.43  0.49  0.67   0.73   1.10   
2020-03-17  0.12  0.19  0.24  0.30  0.47  0.54  0.66  0.91   1.02   1.45   
2020-03-18  0.04  0.02  0.08  0.21  0.54  0.66  0.79  1.08   1.18   1.60   

            30 Yr  
2001-07-31   5.51  
2001-08-01   5.53  
2001-08-02   5.57  
2001-08

### Real Estate Data

Next up is real estate data which we want to import in and do some modifications on.

In [3]:
#Create the real estate data function
def prepare_data_real_estate() -> DataFrame:
    """
    The function for creation of real estate data

    Returns
    -------
    DataFrame
        A dataframe of real estate data

    """
    # Pull in and format the CPI data
    CPI = pd.read_csv("Data/CPI.csv",index_col=0)
    CPI.columns = ["CPI"]
    CPI.index = pd.to_datetime(CPI.index)
    
    # Pull in and format the real estate data
    real_estate = pd.read_csv("Data/Real Estate.csv",index_col=0)
    real_estate.index = pd.to_datetime(real_estate.index)
    real_estate.columns = ["Real Estate"]
    
    # Create the multi-index
    multi_index = pd.MultiIndex.from_tuples(zip(real_estate.index.year,real_estate.index.quarter))
    real_estate.index = multi_index
    quarterly_CPI = CPI.groupby([CPI.index.year, CPI.index.quarter]).last()
    real_estate.index.names = ["Year", "Quarter"]
    quarterly_CPI.index.names = ["Year", "Quarter"]
    
    # Build the data by joining the two
    data = real_estate.join(quarterly_CPI)
    
    return data
real_estate_data = prepare_data_real_estate()
print(real_estate_data)

              Real Estate      CPI
Year Quarter                      
1975 1              59.77   52.800
     2              60.97   53.500
     3              61.18   54.600
     4              62.22   55.600
1976 1              62.90   56.000
...                   ...      ...
2018 4             429.86  252.653
2019 1             434.58  254.095
     2             442.71  255.402
     3             447.87  256.593
     4             451.65  258.444

[180 rows x 2 columns]


## Data Processing

For our report, we are going to actually have it be modular to a given year and quarter to be defined over. One thing we are going to need to know how to do is to grab the first day of the next quarter.

If we want the next quarter we can just add 1, except for 4 where we want that to become 1. If we do the modulo function we can achieve this. Below shows the example of this for all four quarters.

In [4]:
# Grab each next quarter
for quarter in range(1,5):
    print(quarter % 4 + 1)

2
3
4
1


In [5]:
#Define a function that processes the data given the year/quarter we want to use as of

def process_data_yield_curve(yield_curve: DataFrame, year: int, quarter: int) -> DataFrame:
    """
    The function for processing of the yield curve data

    Parameters
    ----------
    yield_curve : DataFrame
        The raw yield curve data
    year : int
        The year for the report
    quarter : int
        The quarter for the report

    Returns
    -------
    DataFrame
        Yield table

    """
    # Copy the yield curve to avoid overwriting
    yield_curve = yield_curve.copy()
    
    #Find the next quarter
    end_quarter = quarter % 4 + 1
    
    #Convert to the month
    end_month = (end_quarter-1) * 3 + 1
    
    # Find the end year which is the same unless the quarter is 4
    end_year = year
    if quarter == 4:
        end_year += 1
    # Finally create the datetime and subtract one day from it for the last day
    end_date = datetime(end_year, end_month, 1) - pd.Timedelta("1D")
    
    # Index the yield 
    yield_curve = yield_curve.loc[:end_date]
    
    # Create the yield table
    yield_table = create_yield_table(yield_curve, end_date)
    
    return yield_table
yield_table = process_data_yield_curve(yield_curve, 2019, 1)
print(yield_table)

       Current  1 Year Ago Curve  5 Years Ago Curve  1 Year Change  \
1 Mo      2.43              1.63               0.03           0.80   
3 Mo      2.40              1.73               0.05           0.67   
6 Mo      2.44              1.93               0.07           0.51   
1 Yr      2.40              2.09               0.13           0.31   
2 Yr      2.27              2.27               0.44           0.00   
3 Yr      2.21              2.39               0.90          -0.18   
5 Yr      2.23              2.56               1.73          -0.33   
7 Yr      2.31              2.68               2.30          -0.37   
10 Yr     2.41              2.74               2.73          -0.33   
20 Yr     2.63              2.85               3.31          -0.22   
30 Yr     2.81              2.97               3.56          -0.16   

       5 Year Change  
1 Mo            2.40  
3 Mo            2.35  
6 Mo            2.37  
1 Yr            2.27  
2 Yr            1.83  
3 Yr            1.31 

Likewise, we build out the same type of function to process our real estate data. It will return to us the returns, the quarterly returns, and the returns adjusted for inflation.

In [6]:
#Define a similar function for the real estate data

def process_data_real_estate(data: DataFrame, year: int, quarter: int) -> Tuple[DataFrame, DataFrame, DataFrame]:
    """
    Function which process the real estate data

    Parameters
    ----------
    data : DataFrame
        Real estate data
    year : int
        The year for the last quarter to use
    quarter : int
        The last quarter to use

    Returns
    -------
    Tuple[DataFrame, DataFrame, DataFrame]
        1. Dataframe of returns
        2. Quarterly returns
        3. Quarterly returns adjusted for inflation

    """
    # Divide all data by the starting values at the end of 2000
    returns = data / data.loc[(1999,4)]
    
    # Only use 2000 and on
    returns = returns.loc[(2000,1):]
    
    # Add adjustments for inflation
    returns["Real Estate Inflation Adjusted"] = returns["Real Estate"] / returns["CPI"]
    
    # Cut the data to be only up to the last quarter/year requested
    returns = returns.loc[:(year, quarter)]
    
    # Get the percentage changes for real estate
    quarterly_returns = data["Real Estate"].pct_change()
    quarterly_returns = quarterly_returns.loc[(2000,1):(year, quarter)]
    quarterly_returns = quarterly_returns.unstack()
    
    # Get the adjusted quarterly returns
    quarterly_returns_adj = returns["Real Estate Inflation Adjusted"].pct_change()
    
    #Because there is no baseline to compare to, the percent change will be negative in the first year
    #We can set it to be equal to the index - 1
    quarterly_returns_adj.iloc[0] = returns["Real Estate Inflation Adjusted"].iloc[0] - 1
    quarterly_returns_adj = quarterly_returns_adj.unstack()
    
    return returns, quarterly_returns, quarterly_returns_adj

# Run our function and check the outputs
returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, 2019, 1)
print(returns_RE.head(5))
print()
print(quarterly_returns_RE.head(5))
print()
print(quarterly_returns_adj_RE.head(5))

              Real Estate       CPI  Real Estate Inflation Adjusted
Year Quarter                                                       
2000 1           1.019145  1.013033                        1.006033
     2           1.035752  1.020142                        1.015302
     3           1.054586  1.028436                        1.025427
     4           1.070837  1.034360                        1.035265
2001 1           1.097240  1.043246                        1.051755

Quarter         1         2         3         4
Year                                           
2000     0.019145  0.016295  0.018183  0.015410
2001     0.024656  0.016759  0.015525  0.012418
2002     0.014362  0.017029  0.019829  0.014168
2003     0.012187  0.012184  0.015411  0.028326
2004     0.015643  0.024409  0.038143  0.021346

Quarter         1         2         3         4
Year                                           
2000     0.006033  0.009213  0.009972  0.009594
2001     0.015928  0.007604  0.013244  0.0

## Generating the Report & Images

We take a step back and begin creation of the report function. Firstly we bring in all data functionality.

In [7]:
def create_report(year: int, quarter: int):
    """
    Function to create the excel economic report

    Parameters
    ----------
    year : int
        The year to use for the last quarter
    quarter : int
        The last quarter to use

    """
    
    # Work through data
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
create_report(2019, 1)

And now to build a function which creates all the different images.

In [8]:
def create_images(yield_table: DataFrame, returns_RE: DataFrame):
    """
    Function to create all the images for use in the report

    Parameters
    ----------
    yield_table : DataFrame
        The yield table
    returns_RE : DataFrame
        The returns data

    """
    
    # Create Treasury Yield Curves image
    graph_data = yield_table[['Current', '1 Year Ago Curve', '5 Years Ago Curve']].copy()
    ax = graph_data.plot(kind="line")
    current_lim = plt.ylim()
    new_lim = [min(current_lim[0],0),current_lim[1]]
    plt.ylim(new_lim)
    plt.xlabel("Maturity")
    plt.ylabel("Yield")
    plt.title("Yield Curves")
    ax.yaxis.set_major_formatter(PercentFormatter())
    plt.savefig("Images/Treasury Yield Curves.png")
    plt.close()
    
    # Create Real Estate vs. Inflation image
    ax = returns_RE[['Real Estate', 'CPI']].plot(kind='line')
    quarter_index = returns_RE.index.map(lambda x: str(x[0]) +"Q"+str(x[1]))
    index_positions = list(range(len(quarter_index)))
    ax.xaxis.set_ticks(index_positions[::4])
    ax.xaxis.set_ticklabels(quarter_index[::4])
    plt.xticks(rotation=70)
    plt.ylabel("Normalized Index Value")
    plt.xlabel("Date")
    plt.title("Normalized Indices")
    plt.savefig("Images/Real Estate vs. Inflation.png", bbox_inches="tight")
    plt.close()
    
    # Create Real Estate Inflation Adjusted image
    ax = returns_RE[["Real Estate", "Real Estate Inflation Adjusted"]].plot(kind='line')
    ax.xaxis.set_ticks(index_positions[::4])
    ax.xaxis.set_ticklabels(quarter_index[::4])
    plt.xticks(rotation=70)
    plt.xlabel("Quarter")
    plt.ylabel("Index Level")
    plt.title("Real Estate Nominal vs. Inflation Adjusted")
    plt.savefig("Images/Real Estate Inflation Adjusted.png", bbox_inches="tight")
    plt.close()

create_images(yield_table, returns_RE)

Then update the create_report function to build the images.

In [9]:
def create_report(year: int, quarter: int):
    """
    Function to create the excel economic report

    Parameters
    ----------
    year : int
        The year to use for the last quarter
    quarter : int
        The last quarter to use

    """
    # Work through data
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    
    # Create images
    create_images(yield_table, returns_RE)
create_report(2019, 1)

## Finalizing the Report

To finalize the report we start adding in the writing of different sheets. First, the treasury sheet.

In [10]:
def create_report(year: int, quarter: int):
    """
    Function to create the excel economic report

    Parameters
    ----------
    year : int
        The year to use for the last quarter
    quarter : int
        The last quarter to use

    """
    # Work through data
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    
    # Create images
    create_images(yield_table, returns_RE)
    
    # Create workbook
    wb = openpyxl.Workbook()
    
    # Grab the active sheet and write the treasury sheet
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    
    # Save workbook
    wb.save("Workbooks/Economic Report.xlsx")
create_report(2019, 1)

We add in a few other things, renaming the title of the worksheet to "Yield Curve", creating a new sheet called "Real Estate", then writing the actual real estate report.

In [11]:
def create_report(year: int, quarter: int):
    """
    Function to create the excel economic report

    Parameters
    ----------
    year : int
        The year to use for the last quarter
    quarter : int
        The last quarter to use

    """
    # Work through data
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    
    # Create images
    create_images(yield_table, returns_RE)
    
    # Create workbook
    wb = openpyxl.Workbook()
    
    # Grab the active sheet and write the treasury sheet
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    
    # Change title
    ws.title = "Yield Curve"
    
    # Add worksheet and write real estate
    ws = wb.create_sheet("Real Estate")
    create_real_estate_report(ws, quarterly_returns_RE, "Images/Real Estate vs. Inflation.png")
    
    # Save workbook
    wb.save("Workbooks/Economic Report.xlsx")
create_report(2019, 1)

We are going to actually take that real estate sheet and replicate it so that we can have on sheet which is our nominal real estate returns, and the other is inflation adjusted!

In [12]:
def create_report(year: int, quarter: int):
    """
    Function to create the excel economic report

    Parameters
    ----------
    year : int
        The year to use for the last quarter
    quarter : int
        The last quarter to use

    """
    # Work through data
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    
    # Create images
    create_images(yield_table, returns_RE)
    
    # Create workbook
    wb = openpyxl.Workbook()
    
    # Grab the active sheet and write the treasury sheet
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    
    # Change title
    ws.title = "Yield Curve"
    
    # Add worksheet and write nominal real estate
    ws = wb.create_sheet("Real Estate Nominal")
    create_real_estate_report(ws, quarterly_returns_RE, "Images/Real Estate vs. Inflation.png")
    
    # Add worksheet to write inflation adjusted real estate
    ws = wb.create_sheet("Real Estate Adjusted")
    create_real_estate_report(ws, quarterly_returns_adj_RE, "Images/Real Estate Inflation Adjusted.png")
    
    # Save workbook
    wb.save("Workbooks/Economic Report.xlsx")
create_report(2019, 1)

One final change, we can actually change the saving to have it name the report with the quarter and year.

In [13]:
def create_report(year: int, quarter: int):
    """
    Function to create the excel economic report

    Parameters
    ----------
    year : int
        The year to use for the last quarter
    quarter : int
        The last quarter to use

    """
    # Work through data
    yield_curve = prepare_data_yield_curve()
    real_estate_data = prepare_data_real_estate()
    yield_table = process_data_yield_curve(yield_curve, year, quarter)
    returns_RE, quarterly_returns_RE, quarterly_returns_adj_RE = process_data_real_estate(real_estate_data, year, quarter)
    
    # Create images
    create_images(yield_table, returns_RE)
    
    # Create workbook
    wb = openpyxl.Workbook()
    
    # Grab the active sheet and write the treasury sheet
    ws = wb.active
    write_treasury_sheet(ws, yield_table)
    
    # Change title
    ws.title = "Yield Curve"
    
    # Add worksheet and write nominal real estate
    ws = wb.create_sheet("Real Estate Nominal")
    create_real_estate_report(ws, quarterly_returns_RE, "Images/Real Estate vs. Inflation.png")
    
    # Add worksheet to write inflation adjusted real estate
    ws = wb.create_sheet("Real Estate Adjusted")
    create_real_estate_report(ws, quarterly_returns_adj_RE, "Images/Real Estate Inflation Adjusted.png")
    
    # Save workbook
    wb.save("Workbooks/Economic Report {}Q{}.xlsx".format(year, quarter))
create_report(2019, 1)

We are finished! You can run this with different year/quarter combinations and have the final report spit out!

In [14]:
#Now run a few times and see how it automatically creates them!
create_report(2019, 4)
create_report(2018, 2)
create_report(2017, 4)